In [2]:
import numpy as np
from xml.dom import minidom
import cv2 as cv  
import os 
import matplotlib.pyplot as plt 
from math import dist
from math import degrees
from math import atan2


In [30]:
class Pitscan():
    def __init__(self, foldername = "", xml_file = ""):
        self.foldername = foldername
        self.xml_file = xml_file
        
    def read_folder(self):
        images = []
        for file in os.listdir(self.foldername)[1:]:
            img_path = os.path.join(self.foldername, file)
            img = cv.imread(img_path, cv.IMREAD_GRAYSCALE)
            images.append(img)
        return images 
    
    def black_white(self, image):
        thresh,binary = cv.threshold(image, 144, 255, cv.THRESH_OTSU)
        return binary
    
    def get_voxel_size(self):
        # applies to only xml files within the Pitscan 
        file = minidom.parse(self.xml_file)
        #use getElementsByTagName() to get tag
        models = file.getElementsByTagName('voxelSize')
        # voxel lengths in mm
        x=float(models[0].attributes['X'].value) 
        y=float(models[0].attributes['Y'].value) 
        z=float(models[0].attributes['Z'].value)
        return x*y*z
    
    def get_volume(self):
        voxel_size = self.get_voxel_size()
        # applies only to folders containing tiff files 
        volume = []
        for file in os.listdir(self.foldername)[1:]:
            ct_slice = os.path.join(self.foldername, file)
            ct_array = cv.imread(ct_slice, cv.IMREAD_GRAYSCALE)
            binarize_image = self.black_white(ct_array)
            contour,_ = cv.findContours(binarize_image, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
            focus, path = self.mask(contour, _, binarize_image)
            volume.append(cv.countNonZero(focus)*voxel_size)
        return np.sum(np.array(volume))
    
    def mask(self, contours, hierarchy, img): 
        max_length = 0 
        position = 0 
        for pos, contour in enumerate(contours): 
            if len(contour) > max_length:
                max_length = len(contour)
                position = pos
        (x,y), radius = cv.minEnclosingCircle(contours[position])
        center = (int(x),int(y))
        blank = np.zeros(img.shape[:2], dtype= "uint8")
        mask = cv.circle(blank, center, int(radius), 255, -1)
        
        return cv.bitwise_and(img, img, mask= mask), contours[position]
    
    def radial_contour(self, img): 
        binarize_slice = self.black_white(img)
        return cv.findContours(binarize_slice, cv.RETR_LIST, cv.CHAIN_APPROX_SIMPLE)
    
    def edges(self, img):
        return cv.Canny(img, 125, 175)
    
    def detect_pit_depth(self):
        pit_depth = []
        for file in os.listdir(self.foldername)[1:]:
            ct_slice = cv.imread(os.path.join(self.foldername, file), cv.IMREAD_GRAYSCALE)
            paths, order = self.radial_contour(ct_slice)
            focus, cont = self.mask(paths, order, self.black_white(ct_slice))
            (x,y), radius = cv.minEnclosingCircle(cont)
            center = (int(x),int(y))
            edges = cv.Canny(focus, 50,150)
            xfit, yfit = np.nonzero(edges)
            xc = np.cos(np.linspace(0, 2 * np.pi, xfit.shape[0])) * (radius+ 50) + center[0]
            yc = np.sin(np.linspace(0, 2 * np.pi, xfit.shape[0])) * (radius + 50) + center[1]

            xfit = xfit - center[0]
            yfit = yfit - center[1]
            xc = xc - center[0]
            yc = yc - center[1]

            thetafit, rhofit = np.arctan2(yfit, xfit), np.hypot(xfit, yfit)
            thetacircle, rhocircle = np.arctan2(yc, xc), np.hypot(xc, yc)
            
            pitdepthslice = np.abs(rhocircle - rhofit)  # mm
            pitdepthslice = pitdepthslice * (self.get_voxel_size() / 1000)  # mm
            pit_depth.extend(pitdepthslice)
            break
        return pit_depth
    
    def filter_close_points(self, points, tolerance=2):
        filtered_points = []
        for i, point1 in enumerate(points):
          for j, point2 in enumerate(points):
            if i != j:
              # Calculate the angular distance between the points
              angle = degrees(atan2(point2[1] - point1[1], point2[0] - point1[0]))
              # Check if the angle is within the tolerance
              if abs(angle) - tolerance <= 0.08 and abs(angle) - tolerance >= 0:
                filtered_points.append(point1)
                break  # Don't need to compare with other points if already found a close one
        return filtered_points
 

    
    def count_pit(self):
        voxel = self.get_voxel_size()
        pit_depth = []
        sizes = []
        location = []
        pitting_array = []
        for file in os.listdir(self.foldername)[1:]:
            ct_slice = cv.imread(os.path.join(self.foldername, file), cv.IMREAD_GRAYSCALE)
            paths, order = self.radial_contour(ct_slice)
            focus, cont = self.mask(paths, order, self.black_white(ct_slice))
            (x,y), radius = cv.minEnclosingCircle(cont)
            center = (int(x),int(y))
            smoothed = cv.GaussianBlur(focus, (5,5), 1)
            edges =cv.Canny(smoothed, 50,150)
            xfit, yfit = np.nonzero(edges)
            radii = []
            gap = []
            selection = []
            points = [(xfit[k],yfit[k]) for k in range(len(xfit))]
            for k in range(xfit.shape[0]):
                point = (xfit[k], yfit[k])
                selection.append(point)
                
            #selected = self.filter_close_points(selection, 2)
        
            for k in range(int(radius)):
                test = int(radius - 1)
                radii.append(test)
                
            for k in range(xfit.shape[0]):
                point = (xfit[k], yfit[k])
                length = dist(point, center)
                gap.append(int(length))
            
            for rad in radii:
                count = 0
                for distance in gap:
                    if rad == distance:
                        count += 1
                        ratio = 2*np.pi*(rad)/count
                        if int(ratio) - 80 <= 10:
                            break
                break
            sizes.append(len(gap))
            loss = rad/radius
            flag = ''
            pit = np.zeros(len(gap))
            diff = np.zeros(len(gap))
            xc = np.cos(np.linspace(0, 2 * np.pi, xfit.shape[0])) * (rad)
            yc = np.sin(np.linspace(0, 2 * np.pi, xfit.shape[0])) * (rad) 
            cont = np.sqrt(xc**2 + yc**2)
            dist_cont = cont - np.array(gap)
            valu = cont - dist_cont
            depth = []
            for k in range(len(valu)):
                if valu[k] < 0.98*rad:
                    flag = True
                    pit[k] = 1
                    for i in range(k,len(valu)):
                        if i != dist_cont.shape[0] -1:
                            a = dist_cont[i]
                            b = dist_cont[i-1]
                            c = dist_cont[i+1]
                            if (1.1*a) < b and (1.1*a) < c: 
                                if a > 0:
                                    depth.append(a)
                                    break        
                        else:
                            a = dist_cont[i]
                            b = dist_cont[i-1]
                            c = dist_cont[0]    
                            if 1.1*a < b and 1.1*a < c: 
                                if a > 0:
                                    depth.append(a)
                                    break                
                else:
                    flag = False 
                    pit[k] = 0
            pitting_array.append(pit)
            pit_depth.append(list(set(depth)))
        
        pitting_matrix = np.vstack([arr [:min(sizes)] for arr in pitting_array ])
        print(pitting_matrix.shape)
        #leftovers = np.vstack([arr[min(sizes):] for arr in pitting_array])
        
        count = 0
        for k in range(pitting_matrix.shape[1]):
            pits = pitting_matrix[:,k]
            for i in range(len(pits)):
                if i == len(pits)-1:
                    if pits[i] != pits[i-1]:
                        count += 0
                else:
                    if pits[i] != pits[i+1]:
                        count += 1
        count /= 2
        sizes.sort()
        return (count, pit_depth, loss, sizes)
         

In [31]:
if __name__ == "__main__": 
    foldername = "/Users/nanao./Desktop/Spyder/images" 
    xml_file = "/Users/nanao./Desktop/unireconstruction copy.xml"
    image_list = Pitscan(foldername, xml_file)
    first_img = image_list.read_folder()[0]
    path, order = image_list.radial_contour(first_img)
    b_w, focus_cont = image_list.mask(path, order, image_list.black_white(first_img))
    voxel = image_list.get_voxel_size()
    volume = image_list.get_volume()
    edges = image_list.edges(b_w)
    x,y = np.nonzero(edges)
    pot_holes = image_list.count_pit()
    print(pot_holes[-1])
    p_w, conti = image_list.mask(path, order, first_img)
   
            

(5, 879)
[879, 931, 961, 980, 1023]
